In this lab, you will implement some of the techniques discussed in the lecture.

Below you are given a solution to the previous scenario. Note that it has two serious drawbacks:
 * The output predictions do not sum up to one (i.e. it does not return a distribution) even though the images always contain exactly one digit.
 * It uses MSE coupled with output sigmoid which can lead to saturation and slow convergence 

**Task 1.** Use softmax instead of coordinate-wise sigmoid and use log-loss instead of MSE. Test to see if this improves convergence. Hint: When implementing backprop it might be easier to consider these two function as a single block and not even compute the gradient over the softmax values. 

**Task 2.** Implement L2 regularization and add momentum to the SGD algorithm. Play with different amounts of regularization and momentum. See if this improves accuracy/convergence.

**Task 3 (optional).** Implement Adagrad, dropout and some simple data augmentations (e.g. tiny rotations/shifts etc.). Again, test to see how these changes improve accuracy/convergence.

**Task 4.** Try adding extra layers to the network. Again, test how the changes you introduced affect accuracy/convergence. As a start, you can try this architecture: [784,100,30,10]


In [1]:
import random
import numpy as np
from torchvision import datasets, transforms

In [2]:
# Let's read the mnist dataset

def load_mnist(path='.'):
    train_set = datasets.MNIST(path, train=True, download=True)
    x_train = train_set.data.numpy()
    _y_train = train_set.targets.numpy()
    
    test_set = datasets.MNIST(path, train=False, download=True)
    x_test = test_set.data.numpy()
    _y_test = test_set.targets.numpy()
    
    x_train = x_train.reshape((x_train.shape[0],28*28)) / 255.
    x_test = x_test.reshape((x_test.shape[0],28*28)) / 255.

    y_train = np.zeros((_y_train.shape[0], 10))
    y_train[np.arange(_y_train.shape[0]), _y_train] = 1
    
    y_test = np.zeros((_y_test.shape[0], 10))
    y_test[np.arange(_y_test.shape[0]), _y_test] = 1

    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_mnist()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [3]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    # Derivative of the sigmoid
    return sigmoid(z)*(1-sigmoid(z))

def softmax(z):
    exps = np.exp(z)
    return exps / exps.sum(axis=0).reshape(1,-1)

In [4]:
class Network(object):
    def __init__(self, sizes):
        # initialize biases and weights with random normal distr.
        # weights are indexed by target node first
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.biases_acc = [np.zeros([y, 1]) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.weights_acc = [np.zeros([y, x]) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, a):
        # Run the network on a batch
        a = a.T
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            a = sigmoid(np.matmul(w, a)+b)

        a = softmax(np.matmul(self.weights[-1], a) + self.biases[-1])
        return a
    
    def update_mini_batch(self, mini_batch, eta, momentum=0, l2=0):
        # Update networks weights and biases by applying a single step
        # of gradient descent using backpropagation to compute the gradient.
        # The gradient is computed for a mini_batch which is as in tensorflow API.
        # eta is the learning rate      
        nabla_b, nabla_w = self.backprop(mini_batch[0].T,mini_batch[1].T)
            
        self.biases_acc = [momentum*b - (eta / len(mini_batch[0])) * nb
                                for b, nb in zip(self.biases_acc, nabla_b)]
        self.biases = [b+ba for b, ba in zip(self.biases, self.biases_acc)]
        self.weights_acc = [momentum*w - (eta / len(mini_batch[0])) * nw
                                for w, nw in zip(self.weights_acc, nabla_w)]
        self.weights = [(1-l2)*w + wa for w, wa in zip(self.weights, self.weights_acc)]
        
    def backprop(self, x, y):
        # For a single input (x,y) return a pair of lists.
        # First contains gradients over biases, second over weights.
        g = x
        gs = [g] # list to store all the gs, layer by layer
        fs = [] # list to store all the fs, layer by layer
        for b, w in zip(self.biases, self.weights):
            f = np.dot(w, g)+b
            fs.append(f)
            g = sigmoid(f)
            gs.append(g)
        # backward pass <- both steps at once
        dLdg = self.cost_derivative(gs[-1], y)
        dLdfs = []
        for w,g in reversed(list(zip(self.weights,gs[1:]))):
            dLdf = np.multiply(dLdg,np.multiply(g,1-g))
            dLdfs.append(dLdf)
            dLdg = np.matmul(w.T, dLdf)
        
        dLdWs = [np.matmul(dLdf,g.T) for dLdf,g in zip(reversed(dLdfs),gs[:-1])] 
        dLdBs = [np.sum(dLdf,axis=1).reshape(dLdf.shape[0],1) for dLdf in reversed(dLdfs)] 
        return (dLdBs,dLdWs)

    def evaluate(self, test_data):
        # Count the number of correct answers for test_data
        pred = np.argmax(self.feedforward(test_data[0]),axis=0)
        corr = np.argmax(test_data[1],axis=1).T
        return np.mean(pred==corr)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations-y) 
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, momentum=0, l2=0, test_data=None):
        x_train, y_train = training_data
        if test_data:
            x_test, y_test = test_data
        for j in range(epochs):
            for i in range(x_train.shape[0] // mini_batch_size):
                x_mini_batch = x_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                y_mini_batch = y_train[(mini_batch_size*i):(mini_batch_size*(i+1))]
                self.update_mini_batch((x_mini_batch, y_mini_batch), eta, momentum, l2)
            if test_data:
                print("Epoch: {0}, Accuracy: {1}".format(j, self.evaluate((x_test, y_test))))
            else:
                print("Epoch: {0}".format(j))


network = Network([784,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.3, momentum=0.9, l2=0.00001, test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.3995
Epoch: 1, Accuracy: 0.432
Epoch: 2, Accuracy: 0.4416
Epoch: 3, Accuracy: 0.5277
Epoch: 4, Accuracy: 0.6068
Epoch: 5, Accuracy: 0.6371
Epoch: 6, Accuracy: 0.7312
Epoch: 7, Accuracy: 0.7363
Epoch: 8, Accuracy: 0.7396
Epoch: 9, Accuracy: 0.7423
Epoch: 10, Accuracy: 0.7434
Epoch: 11, Accuracy: 0.7441
Epoch: 12, Accuracy: 0.7466
Epoch: 13, Accuracy: 0.7474
Epoch: 14, Accuracy: 0.7487
Epoch: 15, Accuracy: 0.7498
Epoch: 16, Accuracy: 0.751
Epoch: 17, Accuracy: 0.7598
Epoch: 18, Accuracy: 0.844
Epoch: 19, Accuracy: 0.8465
Epoch: 20, Accuracy: 0.9179
Epoch: 21, Accuracy: 0.9291
Epoch: 22, Accuracy: 0.9315
Epoch: 23, Accuracy: 0.9331
Epoch: 24, Accuracy: 0.9346
Epoch: 25, Accuracy: 0.9359
Epoch: 26, Accuracy: 0.9377
Epoch: 27, Accuracy: 0.9394
Epoch: 28, Accuracy: 0.9407
Epoch: 29, Accuracy: 0.9414
Epoch: 30, Accuracy: 0.9421
Epoch: 31, Accuracy: 0.9424
Epoch: 32, Accuracy: 0.943
Epoch: 33, Accuracy: 0.9437
Epoch: 34, Accuracy: 0.9444
Epoch: 35, Accuracy: 0.9456
Epoch:

In [5]:
network = Network([784,100,30,10])
network.SGD((x_train, y_train), epochs=100, mini_batch_size=100, eta=0.3, momentum=0.9, l2=0.00001, test_data=(x_test, y_test))

Epoch: 0, Accuracy: 0.5593
Epoch: 1, Accuracy: 0.6413
Epoch: 2, Accuracy: 0.7233
Epoch: 3, Accuracy: 0.8208
Epoch: 4, Accuracy: 0.8293
Epoch: 5, Accuracy: 0.8358
Epoch: 6, Accuracy: 0.9137
Epoch: 7, Accuracy: 0.9195
Epoch: 8, Accuracy: 0.9239
Epoch: 9, Accuracy: 0.9282
Epoch: 10, Accuracy: 0.931
Epoch: 11, Accuracy: 0.9333
Epoch: 12, Accuracy: 0.935
Epoch: 13, Accuracy: 0.9363
Epoch: 14, Accuracy: 0.9377
Epoch: 15, Accuracy: 0.9393
Epoch: 16, Accuracy: 0.9404
Epoch: 17, Accuracy: 0.9417
Epoch: 18, Accuracy: 0.9419
Epoch: 19, Accuracy: 0.9431
Epoch: 20, Accuracy: 0.9441
Epoch: 21, Accuracy: 0.9458
Epoch: 22, Accuracy: 0.9463
Epoch: 23, Accuracy: 0.9464
Epoch: 24, Accuracy: 0.9469
Epoch: 25, Accuracy: 0.9476
Epoch: 26, Accuracy: 0.948
Epoch: 27, Accuracy: 0.9486
Epoch: 28, Accuracy: 0.9489
Epoch: 29, Accuracy: 0.949
Epoch: 30, Accuracy: 0.9499
Epoch: 31, Accuracy: 0.9509
Epoch: 32, Accuracy: 0.9512
Epoch: 33, Accuracy: 0.9516
Epoch: 34, Accuracy: 0.9519
Epoch: 35, Accuracy: 0.9522
Epoch: